In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import math
from pyts.image import RecurrencePlot, GramianAngularField
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, GRU, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model, Sequential
import json
import time
from collections import deque
import warnings
from pickle import dump, load

file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [2]:
# Ignore warning messages
import warnings
warnings.filterwarnings('ignore')

In [3]:
def add_fractal(df, i, look_back=3):
    if i >= look_back and i < df.shape[0] - look_back:
        lows = []
        highs = []

        for j in range(1, look_back + 1):
            prev_mid_low, prev_mid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
            future_mid_low, future_mid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

            lows.append(float(prev_mid_low))
            lows.append(float(future_mid_low))
            highs.append(float(prev_mid_high))
            highs.append(float(future_mid_high))

        mid_low, mid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

        if float(mid_low) < min(lows):
            return float(mid_low), 1.0

        elif float(mid_high) > max(highs):
            return float(mid_high), 0.0

        else:
            return np.nan, np.nan

    else:
        return np.nan, np.nan

def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(closes, periods=14):
    close_delta = closes.diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

def supertrend(barsdata, atr_len=3, mult=3):
    curr_atr = atr(barsdata['Mid_High'], barsdata['Mid_Low'], barsdata['Mid_Close'], lookback=atr_len)
    highs, lows = barsdata['Mid_High'], barsdata['Mid_Low']
    hl2 = (highs + lows) / 2
    final_upperband = upper_band = hl2 + mult * curr_atr
    final_lowerband = lower_band = hl2 - mult * curr_atr

    # initialize Supertrend column to True
    supertrend = [True] * len(df)

    close = barsdata['Mid_Close']
    
    for i in range(1, len(df.index)):
        curr, prev = i, i - 1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True

        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False

        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]

            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

    return supertrend, final_upperband, final_lowerband

def heikin_ashi(opens, highs, lows, closes):
    ha_close = list((opens + highs + lows + closes) / 4)
    ha_opens = []

    opens_list, closes_list = list(opens), list(closes)

    for i in range(len(ha_close)):
        if i == 0:
            ha_opens.append((opens_list[i] + closes_list[i]) / 2)

        else:
            ha_opens.append((ha_opens[i - 1] + ha_close[i - 1]) / 2)

    ha_highs = list(pd.DataFrame({'ha_open': ha_opens, 'ha_close': ha_close, 'high': list(highs)}).max(axis=1))
    ha_lows = list(pd.DataFrame({'ha_open': ha_opens, 'ha_close': ha_close, 'low': list(lows)}).min(axis=1))

    return ha_opens, ha_highs, ha_lows, ha_close

def trend_indicator(opens, highs, lows, closes, ema_period=50, smoothing_period=10):
    ha_open, ha_high, ha_low, ha_close = heikin_ashi(opens, highs, lows, closes)

    ha_o_ema = pd.Series.ewm(pd.DataFrame({'ha_open': ha_open}), span=ema_period).mean()
    ha_h_ema = pd.Series.ewm(pd.DataFrame({'ha_high': ha_high}), span=ema_period).mean()
    ha_l_ema = pd.Series.ewm(pd.DataFrame({'ha_low': ha_low}), span=ema_period).mean()
    ha_c_ema = pd.Series.ewm(pd.DataFrame({'ha_close': ha_close}), span=ema_period).mean()

    return pd.Series.ewm(ha_o_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_h_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_l_ema, span=smoothing_period).mean(), pd.Series.ewm(ha_c_ema, span=smoothing_period).mean()

def qqe_mod(barsdata, rsi_period=6, smoothing=5, qqe_factor=5, qqe2_factor=1.61, threshold=3, mult=0.35, sma_length=50):
    wilders_period = rsi_period * 2 - 1

    curr_rsi = rsi(barsdata, periods=rsi_period)
    rsi_ema = pd.Series.ewm(curr_rsi, span=smoothing).mean()
    atr_rsi = abs(rsi_ema.shift(1) - rsi_ema)
    atr_rsi_ema = pd.Series.ewm(atr_rsi, span=wilders_period).mean()
    dar = pd.Series.ewm(atr_rsi_ema, span=wilders_period).mean() * qqe_factor

    newshortband = rsi_ema + dar
    newlongband = rsi_ema - dar

    rsi_ema_list = list(rsi_ema)

    longband = [0]
    for i in range(1, len(rsi_ema_list)):
        if rsi_ema_list[i - 1] > longband[i - 1] and rsi_ema_list[i] > longband[i - 1]:
            longband.append(max(longband[i - 1],newlongband[i]))

        else:
            longband.append(newlongband[i])

    shortband = [0]
    for i in range(1,len(rsi_ema_list)):
        if rsi_ema_list[i - 1] < shortband[i - 1] and rsi_ema_list[i] < shortband[i - 1]:
            shortband.append(min(shortband[i - 1],newshortband[i]))
            
        else:
            shortband.append(newshortband[i])

    longband = pd.Series(longband)
    shortband = pd.Series(shortband)

    trend = np.where(rsi_ema > longband.shift(1), 1, -1)    
    fastatrrsitl = pd.Series(np.where(trend == 1, longband, shortband))

    basis = (fastatrrsitl - 50).rolling(window=sma_length).mean()
    dev = (fastatrrsitl - 50).rolling(window=sma_length).std() * mult
    upper = basis + dev
    lower = basis - dev

    greenbar1 = rsi_ema - 50 > threshold
    greenbar2 = rsi_ema - 50 > upper
    redbar1 = rsi_ema - 50 < threshold
    redbar2 = rsi_ema - 50 < lower

    # uptrend = np.where((greenbar1 & greenbar2), True, False)
    # downtrend = np.where((redbar1 & redbar2), True, False)

    uptrend = np.where((greenbar2), True, False)
    downtrend = np.where((redbar2), True, False)

    return uptrend, downtrend

def williams_r(highs, lows, closes, length=21, ema_length=15):
    highest_highs = highs.rolling(window=length).max()
    lowest_lows = lows.rolling(window=length).min()

    willy = 100 * (closes - highest_highs) / (highest_highs - lowest_lows)
    willy_ema = pd.Series.ewm(willy, span=ema_length).mean()

    return willy, willy_ema

def squeeze(barsdata, length=20, length_kc=20, mult=1.5):
    # Bollinger bands
    m_avg = barsdata['Mid_Close'].rolling(window=length).mean()
    m_std = barsdata['Mid_Close'].rolling(window=length).std(ddof=0)
    upper_bb = m_avg + mult * m_std
    lower_bb = m_avg - mult * m_std

    # Keltner channel
    tr0 = abs(barsdata['Mid_High'] - barsdata['Mid_Low'])
    tr1 = abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    tr2 = abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    range_ma = tr.rolling(window=length_kc).mean()
    upper_kc = m_avg + range_ma * mult
    lower_kc = m_avg - range_ma * mult

    # Squeeze
    squeeze_on = (lower_bb > lower_kc) & (upper_bb < upper_kc)

    return squeeze_on

In [4]:
def apply_impact(val, impact):
    return float(val) * float(impact)

def clean_value(val):
    return float(val)

def forecast_ratio(forecast, actual):
    if str(forecast) == 'nan' or str(actual) == 'nan':
        return 0.0

    forecast_clean = clean_value(forecast)
    actual_clean = clean_value(actual)
    
    if actual_clean == 0:
        return 0

    return forecast_clean / actual_clean

def previous_ratio(previous, actual):
    if str(previous) == 'nan' or str(actual) == 'nan':
        return 0.0

    previous_clean = clean_value(previous)
    actual_clean = clean_value(actual)
    
    if actual_clean == 0:
        return 0

    return previous_clean / actual_clean

In [5]:
currency_pair = 'Eur_Usd'
years = '2012-2021'

currency1, currency2 = currency_pair.split('_')
currency1, currency2 = currency1.upper(), currency2.upper()

news = pd.read_csv(file_path + 'events.csv')
news.Date = pd.to_datetime(news.Date, utc=True)
news.drop(news[(news['Impact'] != 'low') & (news['Impact'] != 'med') & (news['Impact'] != 'high')].index, inplace=True)
news.loc[news['Impact'] == 'low', 'Impact'] = 1
news.loc[news['Impact'] == 'med', 'Impact'] = 2
news.loc[news['Impact'] == 'high', 'Impact'] = 3
news['Impact'] = pd.to_numeric(news['Impact'])
news['Actual_Class'] = news.apply(lambda row: clean_value(row['Actual']), axis=1)
news['Previous_Class'] = news.apply(lambda row: clean_value(row['Previous']), axis=1)
news_base = news.loc[news['Currency_Code'] == currency1]
news_counter = news.loc[news['Currency_Code'] == currency2]
news_base.drop(['Currency_Code', 'Actual', 'Previous'], axis=1, inplace=True)
news_counter.drop(['Currency_Code', 'Actual', 'Previous'], axis=1, inplace=True)
by_date1 = news_base.groupby('Date')
impact1, actual1, previous1 = by_date1['Impact'].mean().reset_index(), by_date1['Actual_Class'].mean().reset_index(), by_date1['Previous_Class'].mean().reset_index()
news_base = news_base.iloc[0:0]
news_base['Date'], news_base['Impact'], news_base['Actual_Class'], news_base['Previous_Class'] = impact1['Date'], impact1['Impact'], actual1['Actual_Class'], previous1['Previous_Class']
by_date2 = news_counter.groupby('Date')
impact2, actual2, previous2 = by_date2['Impact'].mean().reset_index(), by_date2['Actual_Class'].mean().reset_index(), by_date2['Previous_Class'].mean().reset_index()
news_counter = news_counter.iloc[0:0]
news_counter['Date'], news_counter['Impact'], news_counter['Actual_Class'], news_counter['Previous_Class'] = impact2['Date'], impact2['Impact'], actual2['Actual_Class'], previous2['Previous_Class']

df = pd.read_csv(file_path + f'Oanda_{currency_pair}_H4_{years}.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.reset_index(drop=True, inplace=True)

df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()

df['rsi'] = rsi(df['Mid_Close'])
df['rsi_sma'] = df['rsi'].rolling(50).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['slowk_rsi'], df['slowd_rsi'] = stoch_rsi(df['rsi'])

df['squeeze_on'] = squeeze(df)

df = pd.merge(df, news_base, how='left', on='Date')
df = pd.merge(df, news_counter, how='left', on='Date')
df.reset_index(drop=True, inplace=True)
df = df.fillna(method='ffill')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df['sin_hour'] = np.sin(2 * np.pi * df['Date'].dt.hour / 24)
df['sin_day'] = np.sin(2 * np.pi * df['Date'].dt.day / 7)
df['sin_month'] = np.sin(2 * np.pi * df['Date'].dt.month / 12)

df.drop(['Date', 'Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close'], axis=1, inplace=True)
prices = df[['Bid_High', 'Bid_Low', 'Ask_High', 'Ask_Low']]
scaler = StandardScaler()
df = scaler.fit_transform(df)

dump(scaler, open(f'/Users/mymac/forex_rnn_{currency_pair}_scaler.pickle', 'wb'))

# df.dropna(inplace=True)
# df.reset_index(drop=True, inplace=True)

# df.head()

df

array([[ 2.15471195,  2.2026212 ,  2.18970075, ..., -0.70638514,
         0.53049678,  0.72868085],
       [ 2.21729931,  2.23372152,  2.25490646, ..., -1.4134534 ,
         0.53049678,  0.72868085],
       [ 2.25680312,  2.22351302,  2.24105323, ..., -0.70638514,
         0.53049678,  0.72868085],
       ...,
       [ 1.94957773,  1.93316424,  1.94273112, ...,  0.70775137,
         0.53049678,  0.02500143],
       [ 1.96195241,  1.92532981,  1.8510132 , ..., -0.70638514,
         0.53049678,  0.02500143],
       [ 1.82130934,  1.78905819,  1.82044056, ..., -1.4134534 ,
         0.53049678,  0.02500143]])

In [6]:
prices.head()

,Bid_High,Bid_Low,Ask_High,Ask_Low
0,1.24015,1.23622,1.24031,1.23637
1,1.24146,1.23895,1.24161,1.23908
2,1.24103,1.23837,1.24131,1.23853
3,1.24479,1.24047,1.24495,1.24064
4,1.24589,1.23881,1.24604,1.23897


In [7]:
df.shape

(4569, 28)

In [8]:
look_back_size = 100
train_size = 0.75

In [9]:
sequential_data = []
prev_data = deque(maxlen=look_back_size)

for i in range(len(df) - 1):
    # row = df.iloc[i, :]
    row = df[i, :]
    prev_data.append([val for val in row])

    if len(prev_data) == look_back_size:
        # target = df.loc[df.index[i + 1], ['Bid_High', 'Bid_Low', 'Ask_High', 'Ask_Low']]
        target = prices.loc[prices.index[i + 1], ['Bid_High', 'Bid_Low', 'Ask_High', 'Ask_Low']]
        sequential_data.append([np.array(prev_data), target])

np.random.shuffle(sequential_data)
cutoff_idx = int(len(sequential_data) * train_size)
x_train, y_train, x_test, y_test = [], [], [], []

for i in range(0, cutoff_idx):
    data, target = sequential_data[i]
    x_train.append(data)
    y_train.append(target)

for i in range(cutoff_idx, len(sequential_data)):
    data, target = sequential_data[i]
    x_test.append(data)
    y_test.append(target)

x_train, y_train, x_test, y_test = np.array(x_train).astype('float32'), np.array(y_train).astype('float32'), np.array(x_test).astype('float32'), np.array(y_test).astype('float32')

In [10]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(3351, 100, 28)
(3351, 4)
(1118, 100, 28)
(1118, 4)


In [17]:
# Hyperparameters
n_epochs = 1000
batch_size = 32

# RNN
model = Sequential()

model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(GRU(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(GRU(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(128, activation='relu'))
# model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(4, activation='relu'))

In [18]:
early_stop = EarlyStopping(monitor='mean_squared_error', verbose=1, patience=n_epochs)
model_checkpoint = ModelCheckpoint(f'/Users/mymac/forex_rnn_{currency_pair}', monitor='mean_squared_error', save_best_only=True, verbose=1)

optimizer = Adam()

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mean_squared_error'])

In [19]:
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=n_epochs,
    validation_data=(x_test, y_test),
    callbacks=[early_stop, model_checkpoint]
)

Epoch 1/1000
105/105 [==============================] - ETA: 0s - loss: 0.3102 - mean_squared_error: 0.3102
Epoch 00001: mean_squared_error improved from inf to 0.31021, saving model to /Users/mymac/forex_rnn_Eur_Usd_test


INFO:tensorflow:Assets written to: /Users/mymac/forex_rnn_Eur_Usd_test/assets


INFO:tensorflow:Assets written to: /Users/mymac/forex_rnn_Eur_Usd_test/assets


105/105 [==============================] - 21s 164ms/step - loss: 0.3102 - mean_squared_error: 0.3102 - val_loss: 0.0287 - val_mean_squared_error: 0.0287
Epoch 2/1000
105/105 [==============================] - ETA: 0s - loss: 0.0376 - mean_squared_error: 0.0376
Epoch 00002: mean_squared_error improved from 0.31021 to 0.03764, saving model to /Users/mymac/forex_rnn_Eur_Usd_test


INFO:tensorflow:Assets written to: /Users/mymac/forex_rnn_Eur_Usd_test/assets


...

INFO:tensorflow:Assets written to: /Users/mymac/forex_rnn_Eur_Usd_test/assets


105/105 [==============================] - 14s 130ms/step - loss: 2.1795e-05 - mean_squared_error: 2.1795e-05 - val_loss: 6.8614e-05 - val_mean_squared_error: 6.8614e-05
Epoch 934/1000
105/105 [==============================] - ETA: 0s - loss: 2.4800e-05 - mean_squared_error: 2.4800e-05
Epoch 00934: mean_squared_error did not improve from 0.00002
105/105 [==============================] - 10s 96ms/step - loss: 2.4800e-05 - mean_squared_error: 2.4800e-05 - val_loss: 4.2048e-05 - val_mean_squared_error: 4.2048e-05
Epoch 935/1000
105/105 [==============================] - ETA: 0s - loss: 2.9638e-05 - mean_squared_error: 2.9638e-05
Epoch 00935: mean_squared_error did not improve from 0.00002
105/105 [==============================] - 10s 95ms/step - loss: 2.9638e-05 - mean_squared_error: 2.9638e-05 - val_loss: 4.3806e-05 - val_mean_squared_error: 4.3806e-05
Epoch 936/1000
105/105 [==============================] - ETA: 0s - loss: 2.8517e-05 - mean_squared_error: 2.8517e-05
Epoch 00936: mea

INFO:tensorflow:Assets written to: /Users/mymac/forex_rnn_Eur_Usd_test/assets


INFO:tensorflow:Assets written to: /Users/mymac/forex_rnn_Eur_Usd_test/assets


105/105 [==============================] - 14s 132ms/step - loss: 2.1673e-05 - mean_squared_error: 2.1673e-05 - val_loss: 4.2439e-05 - val_mean_squared_error: 4.2439e-05
Epoch 948/1000
105/105 [==============================] - ETA: 0s - loss: 2.2747e-05 - mean_squared_error: 2.2747e-05
Epoch 00948: mean_squared_error did not improve from 0.00002
105/105 [==============================] - 10s 99ms/step - loss: 2.2747e-05 - mean_squared_error: 2.2747e-05 - val_loss: 3.4060e-05 - val_mean_squared_error: 3.4060e-05
Epoch 949/1000
105/105 [==============================] - ETA: 0s - loss: 2.4046e-05 - mean_squared_error: 2.4046e-05
Epoch 00949: mean_squared_error did not improve from 0.00002
105/105 [==============================] - 10s 99ms/step - loss: 2.4046e-05 - mean_squared_error: 2.4046e-05 - val_loss: 4.6524e-05 - val_mean_squared_error: 4.6524e-05
Epoch 950/1000
105/105 [==============================] - ETA: 0s - loss: 2.3981e-05 - mean_squared_error: 2.3981e-05
Epoch 00950: mea

In [73]:
currency_pair = 'Eur_Usd'
years = '2021-2022'

currency1, currency2 = currency_pair.split('_')
currency1, currency2 = currency1.upper(), currency2.upper()

news = pd.read_csv(file_path + 'events.csv')
news.Date = pd.to_datetime(news.Date, utc=True)
news.drop(news[(news['Impact'] != 'low') & (news['Impact'] != 'med') & (news['Impact'] != 'high')].index, inplace=True)
news.loc[news['Impact'] == 'low', 'Impact'] = 1
news.loc[news['Impact'] == 'med', 'Impact'] = 2
news.loc[news['Impact'] == 'high', 'Impact'] = 3
news['Impact'] = pd.to_numeric(news['Impact'])
news['Actual_Class'] = news.apply(lambda row: clean_value(row['Actual']), axis=1)
news['Previous_Class'] = news.apply(lambda row: clean_value(row['Previous']), axis=1)
news_base = news.loc[news['Currency_Code'] == currency1]
news_counter = news.loc[news['Currency_Code'] == currency2]
news_base.drop(['Currency_Code', 'Actual', 'Previous'], axis=1, inplace=True)
news_counter.drop(['Currency_Code', 'Actual', 'Previous'], axis=1, inplace=True)
by_date1 = news_base.groupby('Date')
impact1, actual1, previous1 = by_date1['Impact'].mean().reset_index(), by_date1['Actual_Class'].mean().reset_index(), by_date1['Previous_Class'].mean().reset_index()
news_base = news_base.iloc[0:0]
news_base['Date'], news_base['Impact'], news_base['Actual_Class'], news_base['Previous_Class'] = impact1['Date'], impact1['Impact'], actual1['Actual_Class'], previous1['Previous_Class']
by_date2 = news_counter.groupby('Date')
impact2, actual2, previous2 = by_date2['Impact'].mean().reset_index(), by_date2['Actual_Class'].mean().reset_index(), by_date2['Previous_Class'].mean().reset_index()
news_counter = news_counter.iloc[0:0]
news_counter['Date'], news_counter['Impact'], news_counter['Actual_Class'], news_counter['Previous_Class'] = impact2['Date'], impact2['Impact'], actual2['Actual_Class'], previous2['Previous_Class']

df = pd.read_csv(file_path + f'Oanda_{currency_pair}_H4_{years}.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.reset_index(drop=True, inplace=True)

df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()

df['rsi'] = rsi(df['Mid_Close'])
df['rsi_sma'] = df['rsi'].rolling(50).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['slowk_rsi'], df['slowd_rsi'] = stoch_rsi(df['rsi'])

df['squeeze_on'] = squeeze(df)

df = pd.merge(df, news_base, how='left', on='Date')
df = pd.merge(df, news_counter, how='left', on='Date')
df.reset_index(drop=True, inplace=True)
df = df.fillna(method='ffill')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df['sin_hour'] = np.sin(2 * np.pi * df['Date'].dt.hour / 24)
df['sin_day'] = np.sin(2 * np.pi * df['Date'].dt.day / 7)
df['sin_month'] = np.sin(2 * np.pi * df['Date'].dt.month / 12)

df.drop(['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close'], axis=1, inplace=True)

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Volume,...,squeeze_on,Impact_x,Actual_Class_x,Previous_Class_x,Impact_y,Actual_Class_y,Previous_Class_y,sin_hour,sin_day,sin_month
0,2021-01-18 10:00:00+00:00,1.20618,1.20750,1.20529,1.20654,1.20628,1.20765,1.20542,1.20667,16317,...,False,2.0,1.0,-1.0,1.0,0.0,0.0,0.5,-0.433884,0.5
1,2021-01-18 14:00:00+00:00,1.20655,1.20801,1.20621,1.20742,1.20669,1.20817,1.20634,1.20756,11937,...,False,1.0,0.0,0.0,1.0,0.0,0.0,-0.5,-0.433884,0.5
2,2021-01-18 18:00:00+00:00,1.20743,1.20810,1.20711,1.20749,1.20758,1.20832,1.20729,1.20797,2965,...,False,1.0,0.0,0.0,1.0,0.0,0.0,-1.0,-0.433884,0.5
3,2021-01-18 22:00:00+00:00,1.20743,1.20963,1.20733,1.20926,1.20794,1.20976,1.20748,1.20941,4537,...,False,1.0,0.0,0.0,1.0,0.0,0.0,-0.5,-0.433884,0.5
4,2021-01-19 02:00:00+00:00,1.20927,1.20979,1.20900,1.20964,1.20943,1.20993,1.20916,1.20978,5138,...,False,1.0,0.0,0.0,1.0,0.0,0.0,0.5,-0.974928,0.5


In [107]:
short_df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_{years}.csv')
short_df.Date = pd.to_datetime(short_df.Date, utc=True)
short_df.reset_index(drop=True, inplace=True)

short_df['ema200'] = pd.Series.ewm(short_df['Mid_Close'], span=200).mean()
short_df['ema100'] = pd.Series.ewm(short_df['Mid_Close'], span=100).mean()
short_df['macd'] = pd.Series.ewm(short_df['Mid_Close'], span=12).mean() - pd.Series.ewm(short_df['Mid_Close'], span=26).mean()
short_df['macdsignal'] = pd.Series.ewm(short_df['macd'], span=9).mean()

short_df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,ema200,ema100,macd,macdsignal
0,2021-04-01 06:00:00+00:00,1.17170,1.17172,1.17141,1.17141,1.17184,1.17187,1.17154,1.17154,1.17177,1.17180,1.17148,1.17148,348,1.171480,1.171480,0.000000,0.000000
1,2021-04-01 06:05:00+00:00,1.17140,1.17182,1.17119,1.17154,1.17153,1.17194,1.17133,1.17167,1.17146,1.17188,1.17126,1.17160,451,1.171540,1.171541,0.000003,0.000001
2,2021-04-01 06:10:00+00:00,1.17153,1.17162,1.17140,1.17161,1.17166,1.17175,1.17151,1.17174,1.17160,1.17168,1.17146,1.17168,324,1.171587,1.171588,0.000006,0.000003
3,2021-04-01 06:15:00+00:00,1.17160,1.17187,1.17153,1.17185,1.17173,1.17200,1.17165,1.17197,1.17166,1.17194,1.17160,1.17191,275,1.171669,1.171671,0.000016,0.000007
4,2021-04-01 06:20:00+00:00,1.17184,1.17253,1.17171,1.17232,1.17196,1.17266,1.17185,1.17244,1.17190,1.17259,1.17178,1.17238,646,1.171814,1.171818,0.000039,0.000017


In [109]:
df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Volume,...,squeeze_on,Impact_x,Actual_Class_x,Previous_Class_x,Impact_y,Actual_Class_y,Previous_Class_y,sin_hour,sin_day,sin_month
1492,2021-12-31 02:00:00+00:00,1.13195,1.13248,1.13087,1.13100,1.13209,1.13263,1.13103,1.13115,1300,...,True,1.0,0.0,0.0,1.0,0.0,0.0,0.5,0.433884,-2.449294e-16
1493,2021-12-31 06:00:00+00:00,1.13101,1.13354,1.13026,1.13279,1.13117,1.13369,1.13041,1.13293,5520,...,True,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.433884,-2.449294e-16
1494,2021-12-31 10:00:00+00:00,1.13278,1.13528,1.13115,1.13475,1.13292,1.13542,1.13130,1.13490,12541,...,True,1.0,0.0,0.0,1.0,0.0,0.0,0.5,0.433884,-2.449294e-16
1495,2021-12-31 14:00:00+00:00,1.13473,1.13857,1.13375,1.13856,1.13488,1.13871,1.13390,1.13870,16274,...,True,1.0,0.0,0.0,1.0,0.0,0.0,-0.5,0.433884,-2.449294e-16
1496,2021-12-31 18:00:00+00:00,1.13855,1.13855,1.13677,1.13680,1.13869,1.13869,1.13745,1.13780,2810,...,True,1.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.433884,-2.449294e-16


In [108]:
short_df.tail()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,ema200,ema100,macd,macdsignal
74778,2022-04-01 05:35:00+00:00,1.10690,1.10694,1.10668,1.10676,1.10707,1.10710,1.10683,1.10691,1.10698,1.10702,1.10676,1.10684,291,1.107953,1.106952,0.000018,-0.000020
74779,2022-04-01 05:40:00+00:00,1.10675,1.10727,1.10672,1.10720,1.10691,1.10744,1.10689,1.10737,1.10683,1.10735,1.10680,1.10728,400,1.107946,1.106958,0.000065,-0.000003
74780,2022-04-01 05:45:00+00:00,1.10721,1.10722,1.10698,1.10704,1.10737,1.10738,1.10715,1.10721,1.10729,1.10730,1.10707,1.10712,183,1.107938,1.106961,0.000087,0.000015
74781,2022-04-01 05:50:00+00:00,1.10703,1.10729,1.10700,1.10726,1.10721,1.10744,1.10717,1.10743,1.10712,1.10736,1.10708,1.10734,227,1.107932,1.106969,0.000122,0.000037
74782,2022-04-01 05:55:00+00:00,1.10727,1.10737,1.10667,1.10673,1.10742,1.10754,1.10683,1.10690,1.10734,1.10745,1.10675,1.10682,439,1.107921,1.106966,0.000106,0.000050


In [138]:
use_pullback = False
pip_movement = 20
divider = 10000 if 'Jpy' not in currency_pair else 100
rounding = 5 if 'Jpy' not in currency_pair else 3
n_bars = 12
pullback_percentage = 0.35
spread_cutoff = 0.10
risk_reward_ratio = 1.5
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
use_nn = True
error = 15

In [139]:
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

reward = 0
n_wins = 0
n_losses = 0
win_streak = 0
loss_streak = 0
curr_win_streak = 0
curr_loss_streak = 0
n_buys = 0
n_sells = 0
pips_risked = []
day_fees = 0
trade = None
lookback = n_bars + 1 if use_pullback else n_bars
lookforward = -1 if use_pullback else 0
pip_movement /= divider
error /= divider

for i in range(lookback, len(short_df)):
    curr_bid_open, curr_bid_high, curr_bid_low, curr_ask_open, curr_ask_high, curr_ask_low, curr_mid_open, curr_date = short_df.loc[short_df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Ask_Open', 'Ask_High', 'Ask_Low', 'Mid_Open', 'Date']]
    ema200, ema100, macd1, macdsignal1 = short_df.loc[short_df.index[i - 1], ['ema200', 'ema100', 'macd', 'macdsignal']]
    macd2, macdsignal2 = short_df.loc[short_df.index[i - 2], ['macd', 'macdsignal']]
    curr_long_df = df.loc[df.Date <= curr_date]
    gte = df.loc[df.Date >= curr_date]

    if len(gte) == 0:
        break

    if len(curr_long_df) < look_back_size + 1:
        continue
    
    spread = abs(curr_ask_open - curr_bid_open)

    mid_opens = list(short_df.loc[short_df.index[i - lookback:i + lookforward], 'Mid_Open'])
    mid_highs = list(short_df.loc[short_df.index[i - lookback:i + lookforward], 'Mid_High'])
    mid_lows = list(short_df.loc[short_df.index[i - lookback:i + lookforward], 'Mid_Low'])
    mid_closes = list(short_df.loc[short_df.index[i - lookback:i + lookforward], 'Mid_Close'])

    # buy_signal = all([mid_opens[j] < mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement
    # sell_signal = all([mid_opens[j] > mid_closes[j] for j in range(len(mid_opens))]) and abs(mid_opens[0] - mid_closes[-1]) >= pip_movement

    # if use_pullback and buy_signal:
    #     mid_open1, mid_high1, mid_low1, mid_close1 = short_df.loc[short_df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
    #     buy_signal = mid_open1 > mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

    # if use_pullback and sell_signal:
    #     mid_open1, mid_high1, mid_low1, mid_close1 = short_df.loc[short_df.index[i - 1], ['Mid_Open', 'Mid_High', 'Mid_Low', 'Mid_Close']]
    #     sell_signal = mid_open1 < mid_close1 and abs(mid_close1 - mid_open1) <= pullback_percentage * abs(mid_high1 - mid_low1)

    macd_vals = [0, macd2, macdsignal2, macd1, macdsignal1]
    buy_signal = ema100 > ema200 and macd2 < macdsignal2 and macd1 > macdsignal1 and max(macd_vals) == 0
    sell_signal = ema100 < ema200 and macd2 > macdsignal2 and macd1 < macdsignal1 and min(macd_vals) == 0

    highest_high, lowest_low = max(mid_highs), min(mid_lows)

    if trade is None:
        if buy_signal:
            open_price = float(curr_ask_open)

            stop_loss = lowest_low
            stop_loss = round(stop_loss, rounding)

            if stop_loss < open_price:
                curr_pips_to_risk = open_price - stop_loss

                if spread <= curr_pips_to_risk * spread_cutoff:
                    stop_gain = round(open_price + (risk_reward_ratio * curr_pips_to_risk), rounding)
                    seq = scaler.transform(curr_long_df.iloc[-101:-1, 1:])
                    bid_high_pred, bid_low_pred, ask_high_pred, ask_low_pred = model.predict(seq.reshape(1, look_back_size, -1))[0]

                    if (bid_low_pred - error > stop_loss and bid_high_pred - error >= stop_gain) or not use_nn:
                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

        elif sell_signal:
            open_price = float(curr_bid_open)
            
            stop_loss = highest_high
            stop_loss = round(stop_loss, rounding)

            if stop_loss > open_price:
                curr_pips_to_risk = stop_loss - open_price

                if spread <= curr_pips_to_risk * spread_cutoff:
                    stop_gain = round(open_price - (risk_reward_ratio * curr_pips_to_risk), rounding)
                    seq = scaler.transform(curr_long_df.iloc[-101:-1, 1:])
                    bid_high_pred, bid_low_pred, ask_high_pred, ask_low_pred = model.predict(seq.reshape(1, look_back_size, -1))[0]

                    if (ask_high_pred + error < stop_loss and ask_low_pred + error <= stop_gain) or not use_nn:
                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, currency_pair)

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, rounding),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)


    if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
        trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
        # trade_amount = -50.0
        reward += trade_amount
        day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

        n_wins += 1 if trade_amount > 0 else 0
        n_losses += 1 if trade_amount < 0 else 0
        curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
        curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

        if curr_win_streak > win_streak:
            win_streak = curr_win_streak

        if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

        trade = None    

    if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
        trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
        # trade_amount = 50.0 * risk_reward_ratio
        reward += trade_amount
        day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

        n_wins += 1 if trade_amount > 0 else 0
        n_losses += 1 if trade_amount < 0 else 0
        curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
        curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

        if curr_win_streak > win_streak:
            win_streak = curr_win_streak

        if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

        trade = None

    if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
        trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
        # trade_amount = -50.0
        reward += trade_amount
        day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

        n_wins += 1 if trade_amount > 0 else 0
        n_losses += 1 if trade_amount < 0 else 0
        curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
        curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

        if curr_win_streak > win_streak:
            win_streak = curr_win_streak

        if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

        trade = None

    if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
        trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
        # trade_amount = 50.0 * risk_reward_ratio
        reward += trade_amount
        day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

        n_wins += 1 if trade_amount > 0 else 0
        n_losses += 1 if trade_amount < 0 else 0
        curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
        curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

        if curr_win_streak > win_streak:
            win_streak = curr_win_streak

        if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

        trade = None

In [140]:
print(reward)
print('Num buys: ' + str(n_sells))
print('Num sells: ' + str(n_buys))
print('Num trades: ' + str(n_buys + n_sells))
print('Num wins: ' + str(n_wins))
print('Num losses: ' + str(n_losses))
print('Win streak: ' + str(win_streak))
print('Loss streak: ' + str(loss_streak))
if len(pips_risked) > 0:
    print('Avg pips risked: ' + str(np.array(pips_risked).mean()))

49.59119000001308
Num buys: 16
Num sells: 3
Num trades: 19
Num wins: 8
Num losses: 11
Win streak: 5
Loss streak: 4
Avg pips risked: 0.0016936842105262976
